# 🧮 Non-Dimensional Incompressible Navier-Stokes Equations (2D, Scalar Form)

#### 🔹 Continuity Equation:
$$
\frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = 0
$$
#### 🔹 x-Momentum Equation:
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} + v \frac{\partial u}{\partial y} = -\frac{\partial p}{\partial x} + Pr \left( \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} \right)
$$
#### 🔹 y-Momentum Equation:
$$
\frac{\partial v}{\partial t} + u \frac{\partial v}{\partial x} + v \frac{\partial v}{\partial y} = -\frac{\partial p}{\partial y} + Pr \left( \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2} \right) + Pr*Ra*T
$$
#### 🔹 Non-Dimensional Heat Equation (with Advection)
$$
\frac{\partial T}{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} =  \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} 
$$

# 🔁 Vorticity–Streamfunction Form (2D, Non-Dimensional)

#### 1. Vorticity Definition:
$$
\omega = -\frac{\partial v}{\partial x} + \frac{\partial u}{\partial y}
$$

$$
\omega = \nabla^2 \psi = \left( \frac{\partial^2 \psi}{\partial x^2} + \frac{\partial^2 \psi}{\partial y^2} \right)
$$

#### 2. Vorticity Transport Equation (from momentum equations):
$$
\frac{\partial \omega}{\partial t} + u \frac{\partial \omega}{\partial x} + v \frac{\partial \omega}{\partial y} = Pr \left( \frac{\partial^2 \omega}{\partial x^2} + \frac{\partial^2 \omega}{\partial y^2} \right) + Pr \cdot Ra \cdot \frac{\partial T}{\partial x}
$$

#### 3. Streamfunction–Vorticity Relation (Poisson Equation):
$$
\nabla^2 \psi = \omega
$$

#### 4. Scalar Heat Equation:
$$
\frac{\partial T}{\partial t} + u \frac{\partial T}{\partial x} + v \frac{\partial T}{\partial y} = \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2}
$$

In [11]:
# Importing dependencies
import numpy as np
# Defining parameters
N = 61 # 121 # Number of points along x axis
dx = 1.0 / (N - 1) # Grid spacing in x direction
M = 61 # 121 # Number of points along y axis
dy = 1.0 / (M - 1) # Grid spacing in y direction
Ra = 3.5e3 # 2.5e4 # Rayleigh number
Pr = 0.7 # Prandtl number
h = 0.01 # Time step

# Initialisation at t=0 with boundary conditions
u = np.zeros((N, M)) # x-velocity
v = np.zeros((N, M)) # y-velocity
T = np.zeros((N, M)) # Temperature
for i in range(N):
    T[i, 0] = 0.5 * np.cos(np.pi * i / (N-1))+1 # Bottom boundary condition of T = 0.5cos(pi*x)+1


# ✨ Creating Vorticity Residual Function
$$
R =  Pr \left( \frac{\partial^2 \omega}{\partial x^2} + \frac{\partial^2 \omega}{\partial y^2} \right) + Pr \cdot Ra \cdot \frac{\partial T}{\partial x} - u \frac{\partial \omega}{\partial x} + v \frac{\partial \omega}{\partial y}
$$


$$
R_{ij} = Pr\left(\frac{\omega_{i-1,j}-2\omega_{i,j}+\omega_{i+1,j}}{\Delta x^2}+\frac{\omega_{i,j-1}-2\omega_{i,j}+\omega_{i,j+1}}{\Delta y^2}\right) + Pr \cdot Ra \cdot \frac{T_{i+1,j} - T_{i-1,j}}{2\Delta x} - u_{i,j} \cdot \frac{\omega_{i+1,j} - \omega_{i-1,j}}{2\Delta x} + v_{i,j} \cdot \frac{\omega_{i,j+1} - \omega_{i,j-1}}{2\Delta y}
$$


In [12]:
def resvor(vor, u, v, dx, dy, Re):
    rvor = np.zeros_like(vor)
    for i in range(1, N - 1):
        for j in range(1, M - 1):
            dvorx2 = (vor[i+1, j] - 2*vor[i, j] + vor[i-1, j]) / dx**2
            dvory2 = (vor[i, j+1] - 2*vor[i, j] + vor[i, j-1]) / dy**2
            dvorx1 = u[i, j] * (vor[i+1, j] - vor[i-1, j]) / (2*dx)
            dvory1 = v[i, j] * (vor[i, j+1] - vor[i, j-1]) / (2*dy)

            rvor[i, j] = (dvorx2 + dvory2) / Re - dvorx1 - dvory1
    return rvor


# ✨ Explicit Euler or Runge-Kutta 4 

## 🔹 Explicit Euler Scheme:
$$
\omega^{n+1}_{i,j} = \omega^n_{i,j} + \Delta t \cdot R^{n}_{i,j}
$$

## 🔹 Explicit Runge-Kutta 4 (RK4) Scheme:
1. Compute intermediate stages:
    $$
    k_1 = R(\omega^n)
    $$
    $$
    k_2 = R\left(\omega^n + \frac{\Delta t}{2} \cdot k_1\right)
    $$
    $$
    k_3 = R\left(\omega^n + \frac{\Delta t}{2} \cdot k_2\right)
    $$
    $$
    k_4 = R\left(\omega^n + \Delta t \cdot k_3\right)
    $$

2. Update:
    $$
    \omega^{n+1} = \omega^n + \frac{\Delta t}{6} \cdot (k_1 + 2k_2 + 2k_3 + k_4)
    $$

In [13]:
def solvor(N, M, dx, dy, Re, h, vor, u, v, method="rk4"):
    if method == "euler":
        rvor = resvor(N, M, vor, u, v, dx, dy, Re)
        vor[1:N-1, 1:M-1] += h * rvor[1:N-1, 1:M-1]
    
    elif method == "rk4":
        vori = np.copy(vor)

        # 1st stage
        rvor = resvor(N, M, vor, u, v, dx, dy, Re)
        vori[1:N-1, 1:M-1] = vor[1:N-1, 1:M-1] + 0.25 * h * rvor[1:N-1, 1:M-1]

        # 2nd stage
        rvor = resvor(N, M, vori, u, v, dx, dy, Re)
        vori[1:N-1, 1:M-1] = vor[1:N-1, 1:M-1] + (h / 3.0) * rvor[1:N-1, 1:M-1]

        # 3rd stage
        rvor = resvor(N, M, vori, u, v, dx, dy, Re)
        vori[1:N-1, 1:M-1] = vor[1:N-1, 1:M-1] + 0.5 * h * rvor[1:N-1, 1:M-1]

        # 4th stage
        rvor = resvor(N, M, vori, u, v, dx, dy, Re)
        vor[1:N-1, 1:M-1] += h * rvor[1:N-1, 1:M-1]
